## All the import

In [1]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping


from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np
import RNA

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle
from keras.layers import Merge

/home/conspiracy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/conspiracy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/conspiracy/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refacto

## Load the data

In [2]:
seqtest = 'augugacugaugcuagugaugucgucgucgaggggaucuagcugaucg'
z = len(seqtest)
np.array(RNA.pfl_fold_up(seqtest,1,z,z))[1:,1]

array([0.99515374, 0.98200108, 0.97327828, 0.83831717, 0.71386684,
       0.70952593, 0.79190253, 0.74114161, 0.24841253, 0.20895404,
       0.37044396, 0.07146792, 0.09764458, 0.13250682, 0.1708176 ,
       0.30276394, 0.90875754, 0.84469873, 0.79897224, 0.34805775,
       0.62012246, 0.22103493, 0.05596614, 0.39760585, 0.50417549,
       0.45270863, 0.77728394, 0.78237052, 0.35248909, 0.27293962,
       0.40199972, 0.82078131, 0.26921011, 0.79933257, 0.63717472,
       0.42815308, 0.4700229 , 0.01334941, 0.25455379, 0.22365579,
       0.20206252, 0.21610497, 0.92423724, 0.77800337, 0.34187502,
       0.1734228 , 0.17031566, 0.79341276])

In [3]:
def traduire(seq):
    nucleotides = ''
    for lettre in seq:
            if lettre=='A' || lettre=='a':
                nucleotides=nucleotides+'a'
            elif lettre=='C' || lettre=='c':
                nucleotides=nucleotides+'c'
            elif lettre=='G' || lettre=='g':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'
    return nucleotides

In [11]:
def convert_sequence(sequence):
    matrice = np.zeros((36,4))
    for i in range(len(sequence)):
            if sequence[i]=='a':
                matrice[i,0] = 1
            elif sequence[i]=='u':
                matrice[i,1] = 1
            elif sequence[i]=='g':
                matrice[i,2] = 1
            elif sequence[i]=='c':
                matrice[i,3] = 1
    return matrice

In [12]:
def convert_matrice3D(seq1,seq2):
    matrice = np.zeros((36,36,1))
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j][0]==1 and seq2[k][1]==1) or (seq1[j][1]==1 and seq2[k][0]==1):
                matrice[j,k,0] = 1
            elif (seq1[j][2]==1 and seq2[k][3]==1) or (seq1[j][3]==1 and seq2[k][2]==1):
                matrice[j,k,0] = 1
            elif (seq1[j][2]==1 and seq2[k][1]==1) or (seq1[j][1]==1 and seq2[k][2]==1):
                matrice[j,k,0] = 1
    return matrice

In [13]:
def get_product_proba(proba1,proba2):
    product = np.zeros((36,36,1))
    for j in range(len(proba1)):
            for k in range(len(proba2)):
                product[j][k] = proba1[j] * proba2[k]
    return product

In [14]:
def get_sum_proba(proba1,proba2):
    somme = np.zeros((36,36,1))
    for j in range(len(proba1)):
            for k in range(len(proba2)):
                somme[j][k] = proba1[j] + proba2[k]
    return somme

In [45]:
def load_datatest():
    datatest = pd.read_csv("genomes/nar-03123-met-g-2015-File010.csv", sep = ",", header=None)
    datatest = np.array(datatest)
    l = len(datatest)
    seq1 = []
    seq2 = []
    matrice3D = []
    product = []
    somme = []
    index = 0
    for i in range(1,l):
        
        begin1 = int(datatest[i,9])
        end1 = int(datatest[i,10])
        size1 = end1-begin1
        if(size1 <= 36):
    
            begin2 = int(datatest[i,11])
            end2 = int(datatest[i,12])
            size2 = end2-begin2
    
            if(size2 <= 36):
                
                print("size 1 : ",size1)
                print("size 2 : ",size2)
                
                proba1 = []
                proba2 = []
            
                sequence = traduire(datatest[i,23][begin1:end1])
                z = len(sequence)
                proba1 = np.array(RNA.pfl_fold_up(sequence,1,z,z))[1:,1]
                seq1.append(convert_sequence(sequence))
                
                sequence = traduire(datatest[i,24][begin2:end2])
                z = len(sequence)
                proba2 = np.array(RNA.pfl_fold_up(sequence, 1, z, z))[1:,1]
                seq2.append(convert_sequence(sequence))
                
                print(index)
                matrice3D.append(convert_matrice3D(seq1[index],seq2[index]))
                product.append(get_product_proba(proba1,proba2))
                somme.append(get_sum_proba(proba1,proba2))
                index+=1
    test = []
    test.append(matrice3D)
    test.append(product)
    test.append(seq1)
    test.append(seq2)
    
    return test

In [50]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

In [51]:
def load_dataset():
    # load your data using this function
    neg1 = []

    neg1 = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    neg1 = np.array(neg1)
    print(len(neg1))
    for i in range(len(neg1)):
        for j in range(2):
            if len(neg1[i,j]) <= 15:
                print(i,"erreur")
            

    neg2 = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    neg2 = np.array(neg2)
    print(len(neg2))
    for i in range(len(neg2)):
        for j in range(2):
            if len(neg2[i,j]) <= 15:
                print(i,"erreur")
    
    neg3 = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    neg3 = np.array(neg3)
    print(len(neg3))
    for i in range(len(neg3)):
        for j in range(2):
            if len(neg3[i,j]) <= 15:
                print(i,"erreur")

    pos1 = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    pos1 = np.array(pos1)
    print(len(pos1))
    for i in range(len(pos1)):
        for j in range(2):
            if len(pos1[i,j]) <= 15:
                print(i,"erreur")
    l = len(pos1)
    
    pos2 = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    pos2 = np.array(pos2)
    print(len(pos2))
    for i in range(len(pos2)):
        for j in range(2):
            if len(pos2[i,j]) <= 15:
                print(i,"erreur")
    l=l+len(pos2)

    pos3 = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    pos3 = np.array(pos3)
    print(len(pos3))
    for i in range(len(pos3)):
        for j in range(2):
            if len(pos3[i,j]) <= 15:
                print(i,"erreur")
    l=l+len(pos3)

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []
    pos3 = []
    neg3 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    l = len(bdd)
    matrice = np.zeros((l,36,36,1))
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1  
    return matrice, matrice1, matrice2, matrice3

In [52]:
def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    data = load_dataset()
    matrice = data[0]
    matrice1 = data[1]
    matrice2 = data[2]
    matrice3 = data[3]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice3[:nb_train])

    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice3[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing

## Calculate the performance

In [53]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

## Create The Model

In [54]:
def get_cnn_network(matrixsize = 6, nbfilter = 24, matrixsize2 = 7, nbfilter2 = 6):    
    k = matrixsize
    # init_weights
    I = np.eye(k)
    M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
    I2 = np.zeros((k,k))
    M2 = np.zeros((k,k))
    for j in range(k):
        I2[:,j] = I[:,k-j-1]
        M2[:,j] = M[:,k-j-1]        
    W = np.zeros((k,k,1,nbfilter))
    W[:,:,0,0] = I
    W[:,:,0,1] = I2
    W[:,:,0,2] = M
    W[:,:,0,3] = M2
    for j in range(4,nbfilter):
        W[:,:,0,j] = np.random.randn(k,k)*0.2
    print('configure cnn network')
 
    model = Sequential()
    model.add(Conv2D(filters = nbfilter, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter)]))    
    model.add(AveragePooling2D(pool_size=(3,3)))
    print(model.output_shape)
    
    k2 = matrixsize2
    I = np.eye(k2)
    M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
    I2=np.zeros((k2,k2))
    M2=np.zeros((k2,k2))
    for j in range(k2):
        I2[:,j] = I[:,k2-j-1]
        M2[:,j] = M[:,k2-j-1]   
    
    Z = np.zeros((k2,k2,nbfilter,nbfilter2))
    
    for u in range(nbfilter):
        Z[:,:,u,0] = I
        Z[:,:,u,1] = I2
        Z[:,:,u,2] = M
        Z[:,:,u,3] = M2
        for p in range(4,nbfilter2):
            Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

    model.add(Conv2D(filters = nbfilter2, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter2)]))
    print(model.output_shape)    
    model.add(Flatten())
        
    return model

In [55]:
def get_cnn_network_seq(nbfilter = 64, kernelsize = 7):        
    model = Sequential()
    model.add(Conv1D(filters = nbfilter, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None)))
    model.add(MaxPooling1D(pool_size=6))
    print(model.output_shape)
    model.add(Flatten())
    return model

In [56]:
def create_model(matrixsize11 = 6, nbfilter11 = 24, matrixsize12 = 7, nbfilter12 = 6, 
                 matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7, nbfilter22 = 6,
                 nbfilter1 = 64, kernel_size1 = 7, nbfilters2 = 64,kernel_size2 = 7,
                 Dense1 = 512, Dense2 = 128, Dense3 = 512, Dense4 = 128, Dense5 = 64):
    training_net=[]
    training_net1 = []
    training_net2 = []
    training_net1.append(get_cnn_network(matrixsize11, nbfilter11, matrixsize12, nbfilter12))
    training_net1.append(get_cnn_network(matrixsize21, nbfilter21, matrixsize22, nbfilter22))
    training_net2.append(get_cnn_network_seq(nbfilter1,kernel_size1))
    training_net2.append(get_cnn_network_seq(nbfilters2,kernel_size2))
    
    
    model2 = Sequential()
    model2.add(Merge(training_net2, mode ='concat'))
    model2.add(Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model2.add(BatchNormalization())
    model2.add(Activation('relu'))
    #model2.add(Dropout(0.1))
    
    model1 = Sequential()
    model1.add(Merge(training_net1, mode ='concat'))
    model1.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    #model1.add(Dropout(0.1))
    model1.add(BatchNormalization())
    model1.add(Activation('relu'))
    
    training_net.append(model1)
    training_net.append(model2)
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
    
    model.add(Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy','roc_curve','auc'])
          
    return model

## Train and Evaluate the model

In [57]:
#Data = load_data(165000,6000,5000)

In [58]:
def train_model(training = [], y = [], batch_size=16, epochs=6, verbose1 = 1, verbose2 = 1,
                              validation = [], val_y = [], testing = [], matrixsize11 = 6, nbfilter11 = 24, 
                              matrixsize12 = 7, nbfilter12 = 6, matrixsize21 = 6, nbfilter21 = 24, matrixsize22 = 7,
                              nbfilter22 = 6, nbfilter1 = 64, kernel_size1 = 7, nbfilters2 = 64,kernel_size2 = 7,
                              Dense1 = 512, Dense2 = 128, Dense3 = 512, Dense4 = 128, Dense5 = 64):
    model = create_model(matrixsize11, nbfilter11, matrixsize12, nbfilter12, matrixsize21, nbfilter21, 
                         matrixsize22, nbfilter22, nbfilter1, kernel_size1, nbfilters2,kernel_size2,
                         Dense1, Dense2, Dense3, Dense4, Dense5)
    earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose = verbose1)
    print('model training')
    model.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    # test   
    
    print('predicting')
    
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("batch_size : ", batch_size)
    print("epochs : ",epochs)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])
    
    return model


## Save The Model

In [59]:
def save_model(model, filename = "model"):
    # serialize model to JSON
    model_json = model.to_json()
    with open(filename+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(filename+".h5")
    json_file.close()
    print("Saved model to disk")

## Load The Model

In [60]:
def load_model(filename = 'model'):
    # load json and create model
    json_file = open(filename+'.json', 'r')
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    json_file.close()
    # load weights into new model
    loaded_model.load_weights(filename+".h5")
    print("Loaded model from disk")
    return loaded_model

In [61]:
def transf(seq1, seq2):
    z = len(seq1)
    proba1 = np.array(RNA.pfl_fold_up(seq1,1,z,z))[1:,1]
    sequ1 = convert_sequence(seq1)
    
    z = len(seq2)
    proba2 = np.array(RNA.pfl_fold_up(seq1,1,z,z))[1:,1]
    sequ2 = convert_sequence(seq2)
    
    matrice3D = convert_matrice3D(sequ1,sequ2)
    product = get_product_proba(proba1,proba2)
       
    test = []
    test.append(matrice3D)
    test.append(product)
    test.append(seq1)
    test.append(seq2)
    
    return test

In [62]:
def get_output(seq1,seq2,window = 36, filename = 'model'):
    model = load_model(filename)
    l1 = len(seq1)
    l2 = len(seq2)
    output = np.zeros((max(1,l1-window),max(1,l2-window)))
    for i in range(max(1,l1-window)):
        for j in range(max(1,l2-window)):
            test = transf(seq1[i:i+35],seq2[j:j+35])
            output[i,j] = (model.predict_proba(test)[:,1])
    
    return output

In [ ]:
def test_on_benchmark(filename = 'model'):
    model = load_model(filename)
    test = load_datatest()
    model.predict_proba(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("batch_size : ", batch_size)
    print("epochs : ",epochs)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

## Run

In [63]:
#model = train_model(Data[0],Data[1],16,6,1,1,Data[2],Data[3],Data[4],6,24,7,6,6,24,7,6,64,7,64,7,512,128,512,128,64)
#save_model(model, "model1")